# fSEND fMRI Run Sequences

In this notebook, I generate .JSON files that will determine how the experimental task will present the stimuli during the scan. The paradigm includes 28 videos from the <a href="https://arxiv.org/abs/1912.05008">Standford Emotional Narratives Dataset</a>, as well as two short, additional videos which involve internal narratives from the main characters of each video. 

The goal of this code is to shuffle all 9 runs such that their overall order varies per participant, and the videos within the runs are shuffled, such that while a run will always show the same 4 individuals, the order will vary.

Additionally, the internal narrative films should also be shuffled, but not intermixed with the SEND videos; participants with even subject IDs will see the narrative films first, and the SEND videos second. 

In [1]:
import random
import pandas as pd
import json
import os
from moviepy.editor import *

In [2]:
out_path = '/Users/f004p74/Documents/dartmouth/projects/fSEND/task/f-send/public/sub_files/'

In [4]:
# Seven SEND runs, 4 videos per run, shuffled within run
send_run1 = ["ID128_vid5","ID117_vid4","ID181_vid6","ID124_vid6"]
send_run2 = ["ID153_vid3","ID131_vid2","ID145_vid4","ID120_vid4"]
send_run3 = ["ID165_vid7","ID112_vid1","ID127_vid3","ID113_vid4"]
send_run4 = ["ID180_vid6","ID170_vid7","ID164_vid3","ID118_vid1"]
send_run5 = ["ID111_vid3","ID116_vid2","ID174_vid2","ID129_vid6"]
send_run6 = ["ID137_vid6","ID179_vid3","ID141_vid1","ID171_vid5"]
send_run7 = ["ID147_vid4","ID123_vid3","ID169_vid2","ID121_vid6"]

send_runs = [send_run1,send_run2,send_run3,send_run4,
            send_run5,send_run6,send_run7]

# One NarratedThought run, shuffled order
narr_version = [[['physical-v1'],['stutterer-v1']],
                [['physical-v2'],['stutterer-v2']]]

for i in range(3,6):
    random.seed(i)
    
    for k in send_runs:
        random.shuffle(k)
        
    for z in narr_version:
        random.shuffle(z)
        
    random.shuffle(send_runs)
    random.shuffle(narr_version)
    
    run_sequence = []
    
    if i % 2 == 0: #Even-numbered subjects watch NarratedThought videos first, SEND second.
        for narr_list in narr_version[0]:
            run_sequence.append(narr_list)
        for sub_list in send_runs:
            run_sequence.append(sub_list)
            
    else: # odd-numbered subjects watch SEND first, NarrThought last. 
        for sub_list in send_runs:
            run_sequence.append(sub_list)
        for narr_list in narr_version[1]:
            run_sequence.append(narr_list)
        
    sequence_dict = {}

    for ix, sub_list in enumerate(run_sequence):
        sequence_dict[str(ix+1)] = {}
        for num,video in enumerate(sub_list):
            sequence_dict[str(ix+1)][sub_list[num]] = {}
    
    for v in sequence_dict.keys():
        for thing in sequence_dict[v]:
            try:
                clip = VideoFileClip("/Users/f004p74/Documents/dartmouth/projects/fSEND/task/f-send/public/stimuli/"+str(thing)+".mp4")
                duration = clip.duration * 1000
            except:
                clip = VideoFileClip("/Users/f004p74/Documents/dartmouth/projects/fSEND/task/f-send/public/stimuli/"+str(thing)+".mp4")
                duration = clip.duration * 1000

            sequence_dict[v][thing] = int(duration)
    
            
    with open(out_path+'sub_'+str(i)+'_run-sequence.json', 'w') as fp:
        json.dump(sequence_dict, fp)

# fSEND post-scan trait impressions

After the participant watches the 30 individuals narrate their personal stories, we will ask them to make trait impressions on each of those individuals, for the following six traits: nosy, bossy, easygoing, rebellious, humble, and conscientious. Faces and trait presentation will be randomized.

In [8]:
img_path = "/Users/f004p74/Documents/dartmouth/projects/fSEND/survey/survey-app/client/public/images"
out_path = "/Users/f004p74/Documents/dartmouth/projects/fSEND/survey/survey-app/client/src/trialOrders/"

In [24]:
imgs = [x.split('.png')[0] for x in os.listdir(img_path)]
imgs = [x for x in imgs if '.DS_Store' not in x]

traits = ['bossy', 'nosy','easygoing','rebellious','humble', 'conscientious']

In [27]:
for i in range(3,71):
    random.seed(i)
    random.shuffle(traits)
    
    img_order_dict = {} 

    for trait in traits:
            img_order_dict[str(trait)] = {}

    for item in img_order_dict.keys():
        random.shuffle(imgs)
        for img in imgs:
            img_order_dict[item][img] = {}        
    
    with open(out_path+'sub_'+str(i)+'_trait-judgements.json', 'w') as fp:
        json.dump(img_order_dict, fp)
